In [6]:
# cargamos un fichero con trazas, usando código de t-mon

%run ProcessxAPISGStatement.py
%run fileBrowserAndUploadButtonToLoadProcessStatements.py
global players_info
players_info= load_players_info_from_file("xapi-sg-sample-data.json")
players_info

Info log (2 lines):
 ... 1st line is valid JSON; interpreting as one-statement-per-line (84 statement(s))
... processed 84/84 statement(s) in 0:00:00.002255. Displaying visualizations ...


In [2]:
# cargamos pandas, plotly

import pandas as pd
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

In [4]:
# montemos un dataframe
df = pd.DataFrame.from_dict(players_info, 'index')
df.loc['John Smith']

TypeError: object of type 'NoneType' has no len()

In [ ]:
# algo de información básica
print(f"total de {len(df)} actores")

total de 5 actores


### Intentemos ver puntuación en cada completable...

In [ ]:
# montamos un dataframe para valores de este gráfico en concreto
cvalues = []
for id, row in df[['completables_scores']].iterrows():
    for k, v in row['completables_scores'].items():
        cvalues.append({'id': id, 'completable': k, 'score': v})
cvalues = pd.DataFrame.from_records(cvalues)

In [ ]:
# mostramos
data = []
for c in cvalues['completable'].unique():
    bar_data = cvalues[cvalues['completable'] == c]
    data.append(go.Bar(x=bar_data['id'], y=bar_data['score'], name=c))    
fig = go.Figure(
    layout_title_text="Puntuación en completables",
    data=data
)
fig.update_xaxes(categoryorder="total descending")
fig.update_layout(barmode='stack')
fig.show()

### Y ahora, veamos progreso en el tiempo, con y sin alinear al inicio del experimento

In [ ]:
# montamos un dataframe para valores de este gráfico en concreto
tvalues = []
for id, row in df[['completables_progress']].iterrows():
    for v in row['completables_progress']['MyFirstGame']:
        tvalues.append({'id': id, 'completable': 'MyFirstGame', 'progress': v[0], 'time': np.datetime64(v[1])})
tvalues = pd.DataFrame.from_records(tvalues)
tvalues

,id,completable,progress,time
0,John Smith,MyFirstGame,0.15,2016-05-24 15:05:49
1,John Smith,MyFirstGame,0.50,2016-05-24 15:15:49
2,John Smith,MyFirstGame,0.70,2016-05-24 15:25:49
3,John Smith,MyFirstGame,0.90,2016-05-24 15:35:49
4,Sarah Connor,MyFirstGame,0.10,2016-05-24 15:05:49
5,Sarah Connor,MyFirstGame,0.35,2016-05-24 15:15:49
6,Sarah Connor,MyFirstGame,0.55,2016-05-24 15:25:49
7,Sarah Connor,MyFirstGame,0.75,2016-05-24 15:35:49
8,James Dean,MyFirstGame,0.45,2016-05-24 15:05:49
9,James Dean,MyFirstGame,0.80,2016-05-24 15:15:49


In [ ]:
# mostramos
data = []
for id in tvalues['id'].unique():
    bar_data = tvalues[tvalues['id'] == id]
    data.append(go.Scatter(x=bar_data['time'], y=bar_data['progress'], name=id, hovertext=f"<b>{id}</b>", mode="lines+markers"))    
fig = go.Figure(
    layout_title_text="Progreso en 1er completable",
    data=data
)
fig.update_xaxes(categoryorder="total descending")
fig.show()

In [ ]:
# y ahora, alineando por 1er evento
data = []
for id in tvalues['id'].unique():
    bar_data = tvalues[tvalues['id'] == id].copy()
    first = bar_data.time.min()
    bar_data.time = pd.to_timedelta(bar_data.time - first) + pd.to_datetime('1970/01/01')
    data.append(go.Scatter(x=bar_data['time'], y=bar_data['progress'], name=id, hovertext=f"<b>{id}</b>", mode="lines+markers"))    
fig = go.Figure(
    layout_title_text="Progreso en 1er completable, desde inicio de sesión",
    data=data
)
fig.update_xaxes(categoryorder="total descending")
fig.show()

In [ ]:
from dash import Dash, html, dash_table, dcc, callback, Output, Input, State
import plotly.subplots as subplots
import fileBrowserAndUploadButtonToLoadProcessStatements

global players_info
# Initialize the app
app = Dash(__name__)

# App layout
app.layout = html.Div([
    html.H1(children='T-Mon'),
    html.Hr(),
    html.H2(children='Select JSON xAPI-SG file to process and see visualisations'),
    dcc.Upload(
        id='upload-data',
        children=html.Div([
            'Drag and Drop or ',
            html.A('Select Files')
        ]),
        style={
            'width': '100%',
            'height': '60px',
            'lineHeight': '60px',
            'borderWidth': '1px',
            'borderStyle': 'dashed',
            'borderRadius': '5px',
            'textAlign': 'center',
            'margin': '10px'
        },
        # Allow multiple files to be uploaded
        multiple=True
    ),
    html.Div(id='output-data-upload'),
    html.Div(id='output-eror', style={'whiteSpace': 'pre-line'}),
    html.Hr(),
    dcc.Tabs(id="t-mon-tabs", value='progress', children=[
        dcc.Tab(label='Progress', value='progress_tab'),
        dcc.Tab(label='Videos', value='video_tab'),
        dcc.Tab(label='Completable', value='completable_tab'),
        dcc.Tab(label='Alternatives', value='alternative_tab'),
        dcc.Tab(label='Interactions', value='interaction_tab'),
        dcc.Tab(label='Accessible', value='interaction_tab'),
        dcc.Tab(label='Menu', value='menu_tab'),
    ]),
    html.H4(children='T-MON, by eUCM research team')
])

@callback(Output('output-data-upload', 'children'),
              Input('upload-data', 'contents'),
              State('upload-data', 'filename'),
              State('upload-data', 'last_modified'))
def update_output(list_of_contents, list_of_names, list_of_dates):
    if list_of_contents is not None:
        return [
            fileBrowserAndUploadButtonToLoadProcessStatements.load_players_info_from_content(c, n, d) for c, n, d in
                zip(list_of_contents, list_of_names, list_of_dates)
        ]
    
if __name__ == '__main__':
    app.run(debug=True, port=8051)

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
Cell In[32], line 55, in update_output(
    list_of_contents=['data:application/json;base64,eyJhY3RvciI6eyJuYW1...sInRpbWVzdGFtcCI6IjIwMTYtMDUtMjRUMTM6MDU6MTJaIn0K'],
    list_of_names=['xapi-sg-sample-data.json'],
    list_of_dates=[1715587109.421]
)
     48 @callback(Output('output-data-upload', 'children'),
     49               Input('upload-data', 'contents'),
     50               State('upload-data', 'filename'),
     51               State('upload-data', 'last_modified'))
     52 def update_output(list_of_contents, list_of_names, list_of_dates):
     53     if list_of_contents is not None:
     54         return [
---> 55             fileBrowserAndUploadButtonToLoadProcessStatements.load_players_info_from_content(c, n, d) for c, n, d in
        c = np.str_('World1-2')
        list_of_contents = ['...']
        list_of_names = ['